## Installing necessary libraries

In [1]:
#Installing necessary libraries
!pip install BeautifulSoup4
!pip install folium
!pip install geocoder

## Importing libraries needed

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import folium
from geopy.geocoders import Nominatim
import numpy as np
from project_lib import Project
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot') # optional: for ggplot-like style
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans 
from scipy.spatial.distance import cdist 
import seaborn as sns

## Data scraping zip codes of cities and municipalities in Iloilo province

In [3]:
url = "https://sites.google.com/site/departmentofphilippines/philippine-zip-codes/provincial-zip-codes/iloilo-zip-codes"

# makes a request to the web page and gets its HTML
r = requests.get(url)

# stores the HTML page in 'soup', a BeautifulSoup object
soup = BeautifulSoup(r.content)

In [4]:
#getting necessary data in the data frame
a = []
Zip_Code_df = pd.DataFrame(columns = ['Location', 'Zip Code', 'Phone Area Code'])
for link in soup.find_all('td'):
    a.append(link.get_text())
    if len(a) == 3:
        df_length = len(Zip_Code_df)
        Zip_Code_df.loc[df_length] = a
        a = []
Zip_Code_df.head()

,Location,Zip Code,Phone Area Code
0,Department of Philippines,Search this site,\n
1,\nThe GovernmentDepartment of Philippines Home...,\n\n \n\nPhilippine ZIP Codes and Area Codes‎ ...,Location\n\n\nZIP Code\n\n\nPhone \n\t\t\t\t\t...
2,Location\n,\nZIP Code\n,\nPhone \n\t\t\t\t\t\t\tArea Code\n
3,\n\t\t\t\t\t\t\tAjuy,\n\t\t\t\t\t\t\t5012,\n\t\t\t\t\t\t\t33
4,\n\t\t\t\t\t\t\tAlimodian,\n\t\t\t\t\t\t\t5028,\n\t\t\t\t\t\t\t33


## Cleaning the data frame

In [5]:
#Cleaning the data frame

#Removing unnecessary texts
Zip_Code_df = Zip_Code_df.replace(['\n','\t','\f'],['','',''], regex=True)

#Removing unnecessary rows and columns
Zip_Code_df = Zip_Code_df.drop(index=range(0,3))
Zip_Code_df = Zip_Code_df.drop(columns='Phone Area Code')

#Reseting the number of indices
Zip_Code_df = Zip_Code_df.reset_index(drop=True)

#Adding address for geocode
for i in range(len(Zip_Code_df['Zip Code'])):
    Zip_Code_df['Zip Code'][i] = Zip_Code_df['Zip Code'][i] + ', Iloilo, Philippines'

Zip_Code_df.head()

,Location,Zip Code
0,Ajuy,"5012, Iloilo, Philippines"
1,Alimodian,"5028, Iloilo, Philippines"
2,Anilao,"5009, Iloilo, Philippines"
3,Badiangan,"5033, Iloilo, Philippines"
4,Balasan,"5018, Iloilo, Philippines"


## Getting the latitude and longitude of each municipality

In [ ]:
# Adding blank columns Latitude and Longitude to Zip_Code_df data frame
Zip_Code_df['Latitude'] = np.nan
Zip_Code_df['Longitude'] = np.nan

# Getting the latitude and longitude of each Location using geocoder api and appending it to the Zip_Code_df data frame
geolocator = Nominatim(user_agent="cal_explorer")

for j in range(len(Zip_Code_df['Zip Code'])):
    location = geolocator.geocode(Zip_Code_df['Zip Code'][j])
    Zip_Code_df['Latitude'][j] = location.latitude
    Zip_Code_df['Longitude'][j] = location.longitude
    
Zip_Code_df

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Visualizing the map of Iloilo municipalities using folium

In [ ]:
# create map of Toronto using latitude and longitude values
map_iloilo = folium.Map(location=[10.6079, 123.8858], zoom_start=10)

# add markers to map
for lat, lng, location in zip(Zip_Code_df['Latitude'], Zip_Code_df['Longitude'], Zip_Code_df['Location']):
    label = folium.Popup(location, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_iloilo)  
    
map_iloilo

## Getting the cluster of Passi City in the revenue count of whole Iloilo province

#### Visualize ranking of Passi City in terms of revenue 

In [ ]:
# initialize list of revenues each municipality in the province of Iloilo
data = [['Ajuy', 114631989.8], ['Alimodian', 101434942.8], ['Anilao', 79750981.88], ['Badiangan', 72392644.72], ['Balasan', 82613191.46], ['Banate', 86704022.88], ['Barotac Nuevo', 127638307.7], ['Barotac Viejo', 111313244.5], ['Batad', 60639876.52], ['Bingawan', 55626622.31], ['Cabatuan', 128571060.5], ['Calinog', 142242009.3], ['Carles', 130473158.4], ['Conception', 112289550.5], ['Dingle', 123694999.4], ['Dueñas', 88459579.17], ['Dumangas', 143496895], ['Estancia', 112382701.2], ['Guimbal', 80622274.83], ['Igbaras', 90632128.53], ['Iloilo City', 2001709094], ['Janiuay', 149819244.8], ['Lambunao', 168550147.1], ['Leganes', 87111715.55], ['Lemery', 80905378.82], ['Leon', 123485719.9], ['Maasin', 94529817.15], ['Miagao', 151140417.9], ['Mina', 68951330.9], ['New Lucena', 66689368.74], ['Oton', 184113453.1], ['Passi', 505723102.9], ['Pavia', 153947939], ['Pototan', 157882222.8], ['San Dionisio', 89934061.87], ['San Enrique', 89111543.67], ['San Joaquin', 143220253.7], ['San Miguel', 77213213.45], ['San Rafael', 55166505.29], ['Santa Barbara', 148077053], ['Sara', 122576063], ['Tigbauan', 129228809.7], ['Tubungan', 70010056.07], ['Zarraga', 73992735.07]
] 
  
# Create the pandas DataFrame 
revenues_df = pd.DataFrame(data, columns = ['Location', 'Revenue']) 
  
# print dataframe. 
revenues_df 

In [ ]:
#Joining Zip_Code_df and revenues_df
Zip_Code_df['Revenue'] = revenues_df['Revenue']
Zip_Code_df

In [ ]:
#Bar plot to check ranking of Passi City in the overall revenue of Iloilo province
bar_df = Zip_Code_df.drop(columns=['Zip Code','Latitude','Longitude'])
bar_df.set_index('Location', inplace=True)
bar_df = bar_df.sort_values('Revenue',ascending=False)

In [ ]:
bar_df.plot(kind='bar', figsize=(20, 10))

plt.ticklabel_format(style='plain', axis='y')

plt.xlabel('Cities/Municipalities') # add to x-label to the plot
plt.ylabel('Revenue') # add y-label to the plot
plt.title('Rank of Cities and Municipalities in Iloilo Province by Revenue') # add title to the plot

plt.show()

## Using Foursquare API for list of venues

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, revenue):
    
    venues_list=[]
    for name, lat, lng, rev in zip(names, latitudes, longitudes, revenue):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            rev,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City/Municipality', 
                  'City/Municipality Latitude', 
                  'City/Municipality Longitude',
                  'City/Municipality Revenue',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

iloilo_venues = getNearbyVenues(names=Zip_Code_df['Location'], latitudes=Zip_Code_df['Latitude'], longitudes=Zip_Code_df['Longitude'], revenue=Zip_Code_df['Revenue'])

In [ ]:
#checking the results
print(iloilo_venues.shape)
iloilo_venues

In [ ]:
#Preparing/cleaning data to be used for K means clustering
K_means_data = pd.DataFrame(iloilo_venues.groupby('City/Municipality').count())
K_means_data = K_means_data.drop(['City/Municipality Latitude', 'City/Municipality Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude'], axis=1)
K_means_data['City/Municipality Revenue'] = [float(x) for x in revenues_df['Revenue']]
K_means_data = K_means_data.reset_index()
K_means_data

In [ ]:
#Preparing/cleaning values for Dependent variable Y to be used for K means clustering
K_means_data_Y = pd.DataFrame(K_means_data['City/Municipality'])
K_means_data_Y

In [ ]:
#Preparing/cleaning values for Independent variable X to be used for K means clustering
K_means_data_X = pd.DataFrame(K_means_data.loc[:,['City/Municipality Revenue', 'Venue Category']])
K_means_data_X

In [ ]:
# Checking the distribution of Y data
sns.distplot(K_means_data_X['Venue Category'])
plt.ticklabel_format(style='plain')

In [ ]:
sns.distplot(K_means_data_X['City/Municipality Revenue'])
plt.ticklabel_format(style='plain')

In [ ]:
# We will normalize the Independent variables, Y using MinMax scaler to ensure that good quality clusters will be generated
scaler = MinMaxScaler() 
K_means_data_X_scaled = pd.DataFrame(scaler.fit_transform(K_means_data_X))
K_means_data_X_scaled.columns = K_means_data_X.columns
K_means_data_X_scaled

In [ ]:
# We will find the best K value using the Elbow Method
distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(K_means_data_X_scaled) 
    kmeanModel.fit(K_means_data_X_scaled)     
      
    distortions.append(sum(np.min(cdist(K_means_data_X_scaled, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / K_means_data_X_scaled.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(K_means_data_X_scaled, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / K_means_data_X_scaled.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

In [ ]:
plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show() 

In [ ]:
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show() 

In [ ]:
# We will also find the best K value using the Silhouette Score Method
from sklearn.metrics import silhouette_score

sil = []
kmax = 10

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(K_means_data_X_scaled)
  labels = kmeans.labels_
  sil.append(silhouette_score(K_means_data_X_scaled, labels, metric = 'euclidean'))

In [ ]:
plt.plot(K, sil, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Silhoutte Score') 
plt.title('The Silhouette Score Method') 
plt.show() 

In [ ]:
# We can see that the clear elbow bend is at k=3 so this is the optimal value
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(K_means_data_X_scaled)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
#Calculating the silhouette score of k=3
print(f'Silhouette Score(k=3): {silhouette_score(K_means_data_X_scaled, kmeans.labels_)}')

In [ ]:
#we will create a new dataframe for merge the input data with the cluster output
K_means_data_cluster = K_means_data.copy()
K_means_data_cluster['Cluster'] = kmeans.labels_
K_means_data_cluster.sort_values(by='Cluster')
K_means_data_cluster[K_means_data_cluster['City/Municipality'] == 'Passi']

In [ ]:
plt.figure(figsize=(15,15))
sns.scatterplot(K_means_data_X_scaled['City/Municipality Revenue'], K_means_data_X_scaled['Venue Category'],hue=K_means_data_cluster['Cluster'], palette='pastel', s=300)
plt.ticklabel_format(style='plain', axis='y')

plt.scatter(kmeans.cluster_centers_[0:, 0], kmeans.cluster_centers_[0:, 1], s=90, c='yellow', label = 'Centroids')
plt.scatter(kmeans.cluster_centers_[1:, 0], kmeans.cluster_centers_[1:, 1], s=90, c='yellow', label = 'Centroids')
plt.scatter(kmeans.cluster_centers_[2:, 0], kmeans.cluster_centers_[2:, 1], s=90, c='yellow', label = 'Centroids')
plt.title('Clusters of Cities/Municipalities')
plt.show()

## Analysis:

<b>Cluster 0 - Low # of known venues and Revenue is less than half of the highest revenue count in the whole Iloilo Province

Cluster 1 - High # of known venues and Revenue is the highest revenue count in the whole Iloilo Province

Cluster 2 - High # of known venues and Revenue is less than half of the highest revenue count in the whole Iloilo Province</b>


Passi City belongs to Cluster 0 where there is low number of known venues and its revenue is less than half of the highest revenue count in the whole Iloilo Province.

Based on the provided output of our model, Passi City's revenue compared to the maximum contributor, Iloilo City, is still very low. 

Given the fact that Iloilo City is the capital of Iloilo province, it is more known to tourists and businessmen. We can assume that one factor can be that the marketing and publicity is better than that of Passi City and it helped increase the revenue of the city. <b>The proof of this assumption is a good consideration for the next milestone of this project.</b>

According to our analysis, to increase the revenue, we can consider improving the tourism sector of Passi City by boosting the marketing and publicity of the city's tourist spots and businesses to attract more tourists and businessmen/investors.

## Suggestion:


With the data we used for this project, we can suggest insights of what kind of business will be known to prospects in Passi City.

In [ ]:
# one hot encoding
iloilo_onehot = pd.get_dummies(iloilo_venues[['Venue Category']], prefix="", prefix_sep="")

# add City/Municipality column back to dataframe
iloilo_onehot['City/Municipality'] = iloilo_venues['City/Municipality'] 

# move City/Municipality column to the first column
fixed_columns = [iloilo_onehot.columns[-1]] + list(iloilo_onehot.columns[:-1])
iloilo_onehot = iloilo_onehot[fixed_columns]

iloilo_onehot

In [ ]:
iloilo_grouped = iloilo_onehot.groupby('City/Municipality').mean().reset_index()
iloilo_grouped

### Here are the top known venue categories in Passi City based on Foursquare API data:

In [ ]:
num_top_venues = 5

passi_top = pd.DataFrame(iloilo_grouped[iloilo_grouped['City/Municipality'] == 'Passi'].T.reset_index())
passi_top.columns = ['venue','freq']
passi_top = passi_top.iloc[1:]
passi_top['freq'] = passi_top['freq'].astype(float)
passi_top = passi_top.round({'freq': 2})
passi_top = passi_top.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)
passi_top

### Here are the top known venue categories in Iloilo City based on Foursquare API data:

In [ ]:
num_top_venues = 20

iloilo_top = pd.DataFrame(iloilo_grouped[iloilo_grouped['City/Municipality'] == 'Iloilo City'].T.reset_index())
iloilo_top.columns = ['venue','freq']
iloilo_top = iloilo_top.iloc[1:]
iloilo_top['freq'] = iloilo_top['freq'].astype(float)
iloilo_top = iloilo_top.round({'freq': 2})
iloilo_top = iloilo_top.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)
iloilo_top

### Here are the top known venue categories in Cluster 1 based on Foursquare API data:

Although the municipalities in Cluster 1 has many number of venues than that of Cluster 0 where Passi City is, we can still consider their top venues.

In [ ]:
# add the cluster in the dataframe
iloilo_grouped_clust = iloilo_grouped.copy()
iloilo_grouped_clust['Cluster'] = K_means_data_cluster['Cluster']
temp = iloilo_grouped_clust[(iloilo_grouped_clust.loc[:,['Cluster']] != 1).all(axis=1)]
iloilo_grouped_clust = iloilo_grouped_clust.drop(temp.index)
iloilo_grouped_clust = iloilo_grouped_clust.drop(columns=['Cluster'])
iloilo_grouped_clust

In [ ]:
for hood in iloilo_grouped_clust['City/Municipality']:
    print("----"+hood+"----")
    passi_top = pd.DataFrame(iloilo_grouped_clust[iloilo_grouped_clust['City/Municipality'] == hood].T.reset_index())
    passi_top.columns = ['venue','freq']
    passi_top = passi_top.iloc[1:]
    passi_top['freq'] = passi_top['freq'].astype(float)
    passi_top = passi_top.round({'freq': 2})
    print(passi_top.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')